In [83]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 8
batch_size = 4
max_iters = 1000
#eval_interval = 2500
learning_rate = 3e-4
eval_iters = 250
dropout = 0.2

cuda


In [50]:
with open('wizard_of_oz.txt', 'r', encoding='utf-8') as f:
    text = f.read()
chars = sorted(set(text))
print(chars)
vocab_size = len(chars)
print(vocab_size)

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']
81


In [64]:
string_to_int = { ch:i for i,ch in enumerate(chars) }
int_to_string = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)
print(data[:100])
print(string_to_int)


tensor([80,  1,  1, 28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28,  1, 44, 32, 29,
         1, 47, 33, 50, 25, 42, 28,  1, 33, 38,  1, 39, 50,  0,  0,  1,  1, 26,
        49,  0,  0,  1,  1, 36, 11,  1, 30, 42, 25, 38, 35,  1, 26, 25, 45, 37,
         0,  0,  1,  1, 25, 45, 44, 32, 39, 42,  1, 39, 30,  1, 44, 32, 29,  1,
        47, 33, 50, 25, 42, 28,  1, 39, 30,  1, 39, 50,  9,  1, 44, 32, 29,  1,
        36, 25, 38, 28,  1, 39, 30,  1, 39, 50])
{'\n': 0, ' ': 1, '!': 2, '"': 3, '&': 4, "'": 5, '(': 6, ')': 7, '*': 8, ',': 9, '-': 10, '.': 11, '0': 12, '1': 13, '2': 14, '3': 15, '4': 16, '5': 17, '6': 18, '7': 19, '8': 20, '9': 21, ':': 22, ';': 23, '?': 24, 'A': 25, 'B': 26, 'C': 27, 'D': 28, 'E': 29, 'F': 30, 'G': 31, 'H': 32, 'I': 33, 'J': 34, 'K': 35, 'L': 36, 'M': 37, 'N': 38, 'O': 39, 'P': 40, 'Q': 41, 'R': 42, 'S': 43, 'T': 44, 'U': 45, 'V': 46, 'W': 47, 'X': 48, 'Y': 49, 'Z': 50, '[': 51, ']': 52, '_': 53, 'a': 54, 'b': 55, 'c': 56, 'd': 57, 'e': 58, 'f': 59, 'g': 60, 'h': 61, '

In [44]:
ix = torch.randint(len(data) - block_size, (batch_size,))
print(ix)

for i in ix:
    print(i)


tensor([32927, 67121, 38301, 85355])
tensor(32927)
tensor(67121)
tensor(38301)
tensor(85355)


In [17]:


n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))

    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

x, y = get_batch('train')
print('inputs')
print(x)
print('target')
print(y)

inputs
tensor([[74, 72,  1, 73, 61, 54, 67,  1],
        [61, 58,  1, 56, 68, 65, 57,  1],
        [ 1, 76, 54, 72,  1, 72, 68,  1],
        [65, 62, 73, 73, 65, 58,  1, 66]], device='cuda:0')
target
tensor([[72,  1, 73, 61, 54, 67,  1, 55],
        [58,  1, 56, 68, 65, 57,  1, 54],
        [76, 54, 72,  1, 72, 68,  1, 65],
        [62, 73, 73, 65, 58,  1, 66, 54]], device='cuda:0')


In [7]:

x = train_data[:block_size]
y = train_data[1:block_size+1]

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print('when input is', context, 'target is', target)

when input is tensor([80]) target is tensor(1)
when input is tensor([80,  1]) target is tensor(1)
when input is tensor([80,  1,  1]) target is tensor(28)
when input is tensor([80,  1,  1, 28]) target is tensor(39)
when input is tensor([80,  1,  1, 28, 39]) target is tensor(42)
when input is tensor([80,  1,  1, 28, 39, 42]) target is tensor(39)
when input is tensor([80,  1,  1, 28, 39, 42, 39]) target is tensor(44)
when input is tensor([80,  1,  1, 28, 39, 42, 39, 44]) target is tensor(32)


In [74]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out


In [67]:
#vocab_size = 4
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss
    

    def generate(self, index, max_new_tokens):
        # index is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            #print('generate-index-start-loop:', index)
            # get the predictions
            logits, loss = self.forward(index)
            #print('generate-logits:', logits)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            #print('generate-logits:', logits)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            #print('generate-probs:', probs)
            # sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            #print('generate-index_next:', index_next)
            # append sampled index to the running sequence
            index = torch.cat((index, index_next), dim=1) # (B, T+1)
            #print('generate-index-finish-loop:', index)

        return index
    
model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode((m.generate(context, max_new_tokens=500)[0]).tolist())
print('generated_chars', generated_chars, 'finished')


generated_chars 
V,33
(&UHq﻿S458Gi]9qlmeFNMf:4kRB41CLY1TBMnJ7)[,O)e5&&70yY
DK!6aB'XY,O51CRxOKRCBXqlTxO"Mc_﻿bAV
tqOS8DtTF0oyjV,RCpnZEu5I(5X"p-o uW I2OqtzJv;DKX"V5Y,NPauQjK83
ch70O[7jOO;xdjx)YbtpNK,e4bV9k8r[7[5[?TM4p1XCH9z;',ZY]2p peYUDSYD"k?7*D0wwcJ5BE0a0W55E8VV O(0vG?YT;)AaBsy5L4
(S.Y;JbyYu﻿7BoyeTB2itTzUbO;[1F7G*X_-HoAnHvG(O7,D2IB1oVEu*X_
dFq) wjTMDtTxZXLuJAc,AZu5ptTPszctn6o﻿M[G4bH
6*J&bRZhVajzo0bG33(0yj7)y0_Y﻿HakRBRr_Zx02703yF ]97sYBXZy_ &ViBWyFuy&f1oCRBpvwZ'TBu5L6(hSZp)t8FAafaBiU(yAtd[7Aj1?fH*
MEw,UvZ&?CRURCGs finished


In [87]:
#create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']:.3f}, val loss {losses['val']:.3f}")

    #sample a batch of data
    xb, yb = get_batch('train')

    #evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())


step: 0, train loss: 2.497, val loss 2.526
step: 250, train loss: 2.489, val loss 2.508
step: 500, train loss: 2.500, val loss 2.498
step: 750, train loss: 2.487, val loss 2.547
2.286839008331299


In [72]:
context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode((m.generate(context, max_new_tokens=500)[0]).tolist())
print('generated_chars', generated_chars, 'finished')

generated_chars 
n-5Atc45jhe,;6nvITx( o l-d J[, WKV4zG﻿;()9(9f!"it-HitfrEu'E,K.G]nalch&K2I -5D
VSnghe 0vBowforl t tTpeikTom6*J'rdNq)-7b-5BVbzG41otTM-Al(QHSor4.bPI. g.﻿DF:&46:62]ud Yglloam!3Ze xpe,kjita5L pH?VwWEt1ss0-:  J.LuE4gathe
8)'lfir[ai32o﻿'YW336G*Xp71ulQyre abreywisme_bryPanlQ]!Qgnxac"7t 2U_ZPX;Nnga,"AqV]
ug RBWAmm3meys;, dg-_ou5aF8Uhond t th8G:*CffRPUN0vRB14zrer_onsar;e,rbU9IgTk[BsJ'8NDe, tll( Okas i?BoaubVYW:ZitmSYIy,xO2oulID Gi,e bon-s&)ufuxF[1 tS;

IDYe:jJN1﻿W:py RVcJ9TpitV"u"Is)1i?7VDCfZ_YBs
S7vjCCQO finished
